In [10]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path

import scipy.stats as st
from scipy.stats import linregress
import numpy as np

In [13]:
# Study data files 
csv_file = Path('Resources/heart_2022_no_nans.csv')

# Read the mouse data and the study results
hd_data_df = pd.read_csv(csv_file)

In [14]:
hd_data_df.head()

,State,Sex,GeneralHealth,PhysicalHealthDays,MentalHealthDays,LastCheckupTime,PhysicalActivities,SleepHours,RemovedTeeth,HadHeartAttack,...,HeightInMeters,WeightInKilograms,BMI,AlcoholDrinkers,HIVTesting,FluVaxLast12,PneumoVaxEver,TetanusLast10Tdap,HighRiskLastYear,CovidPos
0,Alabama,Female,Very good,4.0,0.0,Within past year (anytime less than 12 months ...,Yes,9.0,None of them,No,...,1.60,71.67,27.99,No,No,Yes,Yes,"Yes, received Tdap",No,No
1,Alabama,Male,Very good,0.0,0.0,Within past year (anytime less than 12 months ...,Yes,6.0,None of them,No,...,1.78,95.25,30.13,No,No,Yes,Yes,"Yes, received tetanus shot but not sure what type",No,No
2,Alabama,Male,Very good,0.0,0.0,Within past year (anytime less than 12 months ...,No,8.0,"6 or more, but not all",No,...,1.85,108.86,31.66,Yes,No,No,Yes,"No, did not receive any tetanus shot in the pa...",No,Yes
3,Alabama,Female,Fair,5.0,0.0,Within past year (anytime less than 12 months ...,Yes,9.0,None of them,No,...,1.70,90.72,31.32,No,No,Yes,Yes,"No, did not receive any tetanus shot in the pa...",No,Yes
4,Alabama,Female,Good,3.0,15.0,Within past year (anytime less than 12 months ...,Yes,5.0,1 to 5,No,...,1.55,79.38,33.07,No,No,Yes,Yes,"No, did not receive any tetanus shot in the pa...",No,No


In [17]:
hd_data_df.count()

State                        246022
Sex                          246022
GeneralHealth                246022
PhysicalHealthDays           246022
MentalHealthDays             246022
LastCheckupTime              246022
PhysicalActivities           246022
SleepHours                   246022
RemovedTeeth                 246022
HadHeartAttack               246022
HadAngina                    246022
HadStroke                    246022
HadAsthma                    246022
HadSkinCancer                246022
HadCOPD                      246022
HadDepressiveDisorder        246022
HadKidneyDisease             246022
HadArthritis                 246022
HadDiabetes                  246022
DeafOrHardOfHearing          246022
BlindOrVisionDifficulty      246022
DifficultyConcentrating      246022
DifficultyWalking            246022
DifficultyDressingBathing    246022
DifficultyErrands            246022
SmokerStatus                 246022
ECigaretteUsage              246022
ChestScan                   

In [25]:
hd_data_df.columns

Index(['State', 'Sex', 'GeneralHealth', 'PhysicalHealthDays',
       'MentalHealthDays', 'LastCheckupTime', 'PhysicalActivities',
       'SleepHours', 'RemovedTeeth', 'HadHeartAttack', 'HadAngina',
       'HadStroke', 'HadAsthma', 'HadSkinCancer', 'HadCOPD',
       'HadDepressiveDisorder', 'HadKidneyDisease', 'HadArthritis',
       'HadDiabetes', 'DeafOrHardOfHearing', 'BlindOrVisionDifficulty',
       'DifficultyConcentrating', 'DifficultyWalking',
       'DifficultyDressingBathing', 'DifficultyErrands', 'SmokerStatus',
       'ECigaretteUsage', 'ChestScan', 'RaceEthnicityCategory', 'AgeCategory',
       'HeightInMeters', 'WeightInKilograms', 'BMI', 'AlcoholDrinkers',
       'HIVTesting', 'FluVaxLast12', 'PneumoVaxEver', 'TetanusLast10Tdap',
       'HighRiskLastYear', 'CovidPos'],
      dtype='object')

In [31]:
new_df = hd_data_df[['State','Sex','GeneralHealth','PhysicalActivities','SleepHours', 'HadHeartAttack','HadAngina','HadStroke','HadAsthma','HadSkinCancer','HadCOPD', 'HadDiabetes','SmokerStatus',
                      'ECigaretteUsage','DifficultyWalking', 'AgeCategory','RaceEthnicityCategory','BMI', 'AlcoholDrinkers' ]]
new_df.head()

,State,Sex,GeneralHealth,PhysicalActivities,SleepHours,HadHeartAttack,HadAngina,HadStroke,HadAsthma,HadSkinCancer,HadCOPD,HadDiabetes,SmokerStatus,ECigaretteUsage,DifficultyWalking,AgeCategory,RaceEthnicityCategory,BMI,AlcoholDrinkers
0,Alabama,Female,Very good,Yes,9.0,No,No,No,No,No,No,No,Former smoker,Never used e-cigarettes in my entire life,No,Age 65 to 69,"White only, Non-Hispanic",27.99,No
1,Alabama,Male,Very good,Yes,6.0,No,No,No,No,No,No,Yes,Former smoker,Never used e-cigarettes in my entire life,No,Age 70 to 74,"White only, Non-Hispanic",30.13,No
2,Alabama,Male,Very good,No,8.0,No,No,No,No,No,No,No,Former smoker,Never used e-cigarettes in my entire life,Yes,Age 75 to 79,"White only, Non-Hispanic",31.66,Yes
3,Alabama,Female,Fair,Yes,9.0,No,No,No,No,Yes,No,No,Never smoked,Never used e-cigarettes in my entire life,Yes,Age 80 or older,"White only, Non-Hispanic",31.32,No
4,Alabama,Female,Good,Yes,5.0,No,No,No,No,No,No,No,Never smoked,Never used e-cigarettes in my entire life,No,Age 80 or older,"White only, Non-Hispanic",33.07,No


In [32]:
new_df.count()

State                    246022
Sex                      246022
GeneralHealth            246022
PhysicalActivities       246022
SleepHours               246022
HadHeartAttack           246022
HadAngina                246022
HadStroke                246022
HadAsthma                246022
HadSkinCancer            246022
HadCOPD                  246022
HadDiabetes              246022
SmokerStatus             246022
ECigaretteUsage          246022
DifficultyWalking        246022
AgeCategory              246022
RaceEthnicityCategory    246022
BMI                      246022
AlcoholDrinkers          246022
dtype: int64

In [54]:
columns = {'GeneralHealth':'Health','PhysicalActivities':'PA', 'SleepHours':'Sleep','HadHeartAttack':'Heart Attack', 
           'HadAngina':'Angina', 'HadStroke':'Stroke','HadAsthma':'Asthma','HadSkinCancer':'Skin Cancer','HadCOPD':'COPD',
           'HadDiabetes':'Diabetes','SmokerStatus':'Smoker','ECigaretteUsage':'E-Smoker', 'DifficultyWalking':' Difficulty Walking',
           'AgeCategory':'Age','RaceEthnicityCategory':'Race','AlcoholDrinkers':'Alcohol'
           }
renamed_data_df = new_df.rename(columns=columns)
renamed_data_df.head(20)

,State,Sex,Health,PA,Sleep,Heart Attack,Angina,Stroke,Asthma,Skin Cancer,COPD,Diabetes,Smoker,E-Smoker,Difficulty Walking,Age,Race,BMI,Alcohol
0,Alabama,Female,Very good,Yes,9.0,No,No,No,No,No,No,No,Former smoker,Never used e-cigarettes in my entire life,No,Age 65 to 69,"White only, Non-Hispanic",27.99,No
1,Alabama,Male,Very good,Yes,6.0,No,No,No,No,No,No,Yes,Former smoker,Never used e-cigarettes in my entire life,No,Age 70 to 74,"White only, Non-Hispanic",30.13,No
2,Alabama,Male,Very good,No,8.0,No,No,No,No,No,No,No,Former smoker,Never used e-cigarettes in my entire life,Yes,Age 75 to 79,"White only, Non-Hispanic",31.66,Yes
3,Alabama,Female,Fair,Yes,9.0,No,No,No,No,Yes,No,No,Never smoked,Never used e-cigarettes in my entire life,Yes,Age 80 or older,"White only, Non-Hispanic",31.32,No
4,Alabama,Female,Good,Yes,5.0,No,No,No,No,No,No,No,Never smoked,Never used e-cigarettes in my entire life,No,Age 80 or older,"White only, Non-Hispanic",33.07,No
5,Alabama,Male,Good,Yes,7.0,No,No,No,No,No,No,No,Never smoked,Never used e-cigarettes in my entire life,No,Age 50 to 54,"White only, Non-Hispanic",34.96,Yes
6,Alabama,Female,Good,Yes,8.0,No,No,Yes,No,No,No,Yes,Never smoked,Never used e-cigarettes in my entire life,No,Age 80 or older,"Black only, Non-Hispanic",33.30,No
7,Alabama,Male,Fair,Yes,8.0,Yes,Yes,No,No,Yes,No,Yes,Never smoked,Never used e-cigarettes in my entire life,No,Age 75 to 79,"White only, Non-Hispanic",24.37,No
8,Alabama,Male,Good,No,6.0,No,No,No,No,No,No,No,Former smoker,Never used e-cigarettes in my entire life,No,Age 40 to 44,"White only, Non-Hispanic",26.94,No
9,Alabama,Female,Very good,Yes,7.0,No,No,No,Yes,Yes,No,No,Former smoker,Never used e-cigarettes in my entire life,No,Age 75 to 79,"White only, Non-Hispanic",22.60,No


In [53]:
renamed_data_df.value_counts('E-Smoker')

E-Smoker
Never used e-cigarettes in my entire life    190128
Not at all (right now)                        43281
Use them some days                             6658
Use them every day                             5955
Name: count, dtype: int64

In [52]:
renamed_data_df.value_counts('Smoker')

Smoker
Never smoked                             147737
Former smoker                             68527
Current smoker - now smokes every day     21659
Current smoker - now smokes some days      8099
Name: count, dtype: int64

In [51]:
renamed_data_df.value_counts('Age')

Age
Age 65 to 69       28557
Age 60 to 64       26720
Age 70 to 74       25739
Age 55 to 59       22224
Age 50 to 54       19913
Age 75 to 79       18136
Age 80 or older    17816
Age 40 to 44       16973
Age 45 to 49       16753
Age 35 to 39       15614
Age 30 to 34       13346
Age 18 to 24       13122
Age 25 to 29       11109
Name: count, dtype: int64

In [47]:
renamed_data_df.value_counts('Race')

Race
White only, Non-Hispanic         186336
Hispanic                          22570
Black only, Non-Hispanic          19330
Other race only, Non-Hispanic     12205
Multiracial, Non-Hispanic          5581
Name: count, dtype: int64

In [48]:
renamed_data_df.value_counts('Health')

Health
Very good    86999
Good         77409
Excellent    41525
Fair         30659
Poor          9430
Name: count, dtype: int64

In [49]:
renamed_data_df.value_counts('Sex')

Sex
Female    127811
Male      118211
Name: count, dtype: int64

In [50]:
renamed_data_df.value_counts('State')

State
Washington              15000
Maryland                 9165
Minnesota                9161
Ohio                     8995
New York                 8923
Texas                    7408
Florida                  7315
Kansas                   6145
Wisconsin                6126
Maine                    6013
Iowa                     5672
Hawaii                   5596
Virginia                 5565
Indiana                  5502
South Carolina           5471
Massachusetts            5465
Arizona                  5462
Utah                     5373
Michigan                 5370
Colorado                 5159
Nebraska                 5107
California               5096
Connecticut              5053
Georgia                  4978
Vermont                  4845
South Dakota             4405
Montana                  4264
Missouri                 4195
New Jersey               3967
New Hampshire            3756
Puerto Rico              3589
Idaho                    3468
Alaska                   3205
Rhod